# How to process PDFs?

There is a very nice package PyMuPDF, but unfortunately it is AGPL
https://pymupdf.readthedocs.io/en/latest/about.html#license-and-copyright


Instead I'll use pypdf

In [ ]:
%pip install pypdf --quiet

In [1]:
import pypdf
import os
# read pdfs

directory_path = r"C:\Users\kryst\Documents\Stuff"

pdfs = []

# recursively find all pdfs in directory

for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

print(pdfs)

['C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Application Security\\Security_slides08_Apps.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Crypto for Cybersecurity\\Security_slides03_Crypto.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Firewalls\\Security_slides07_Firewall.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Fundamentals of Cybersecurity\\Security_slides02_Basics.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Higher-Security Environments\\Security_slides10_HighSec.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Introduction\\Security_slides01_Intro.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Network Security\\Security_slides05_Network.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity\\Operating Systems Security\\Security_slides04_OS.pdf', 'C:\\Users\\kryst\\Documents\\Stuff\\University\\Cybersecurity

In [47]:
%pip install cryptography --quiet --upgrade

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\kryst\appdata\local\programs\python\python311\lib\site-packages\wordcloud-1.8.2.post4+g5dd8d3e-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
all_pages = []

for pdf in pdfs:
    # read pdf page by page
    pdf_file = open(pdf, 'rb')
    pdf_reader = pypdf.PdfReader(pdf_file)
    # get number of pages
    for i, page in enumerate(pdf_reader.pages):
        
        content = page.extract_text()
        all_pages.append(
            {
                "path": pdf,
                "page": i,
                "content": content
            }
        )

# dump into jsonl
import json

with open("pdfs.jsonl", "w") as f:
    for page in all_pages:
        f.write(json.dumps(page))
        f.write("\n")
print(len(all_pages))

1443


In [3]:
# https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
def get_approx_tokens(sequence):
    return len(sequence) / 4

In [4]:
from nltk import sent_tokenize

def sliding_window(seq, n=512):
    """Returns a sliding window (of width n) over data from the iterable"""
    seq = sent_tokenize(seq)

    # sentences should not be broken up
    
    lengths = []
    for sentence in seq:
        sentence_token_len = get_approx_tokens(sentence)
        lengths.append(sentence_token_len)

    result = []
    start = 0
    for i in range(len(lengths)+1):
        if sum(lengths[start:i]) > n:
            result.append(seq[start:i-1])
            start +=1

        # last sentence
    result.append(seq[start:])
    return result    



# list(sliding_window("123456. 12345678. 123456789."))

In [5]:
passages = []

for page in all_pages:


    # split page into passages
    content = page["content"]

    content_split = sliding_window(content)

    passages.extend(
        [
            {
                "path": page["path"],
                "page": page["page"],
                "passage": passage
            }
            for passage in content_split
        ]
    )

print(len(passages))

1455


In [6]:
from fastembed.embedding import FlagEmbedding as Embedding
from tqdm import tqdm

model = Embedding(model_name="BAAI/bge-base-en-v1.5", max_length=512)


In [9]:

# embed passages

embeddings = list(model.passage_embed([passage["passage"] for passage in passages]))

for i, embedding in enumerate(embeddings):

    passages[i]["embedding"] = embedding


In [10]:
# save embeddings

with open("passages.jsonl", "w") as f:
    for passage in passages:
        passage["embedding"] = passage["embedding"].tolist()
        f.write(json.dumps(passage))
        f.write("\n")

# What to do if we can't extract text?

## TesseractOCR

In [1]:
%pip install pytesseract --quiet

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\kryst\appdata\local\programs\python\python311\lib\site-packages\wordcloud-1.8.2.post4+g5dd8d3e-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Install tesseract executable - https://tesseract-ocr.github.io/tessdoc/Installation.html

In [3]:
from PIL import Image

import pytesseract


pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [13]:
# download jpg

import requests

url = "https://pomoc.ifirma.pl/wp-content/uploads/2022/11/paragon2-1.jpg"

r = requests.get(url, allow_redirects=True)

open('paragon.jpg', 'wb').write(r.content)


511142

In [14]:
import cv2

img_cv = cv2.imread(r'paragon.jpg')

# By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
# we need to convert from BGR to RGB format/mode:
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
print(pytesseract.image_to_string(img_rgb))

LEROY-NERLIN POLSKA SP. 2 0.0.
Ul. Burakowska 14, 01-066 Warszaua
SKLEP LEROY MERLIN WROCtAU)
53-015 WROCKAY, Al, Karkonoska 85
53-015 Wroctay
Karkonoska 865
NIP 113-00-89-950

n
PARAGON FISKALNY
82680523 BAT ZLEW ELASTIC CZARNY ELAST

1
43471624 ELEKTR OGRZEWACZ DAFI 4,5KW +P
1 x

r wydr. 176667/0622

129,00 129,00 A
199,00 199,00 8

Sprzed.’aped. PiU fh ts aaa ae 328,00
Kwota PTU A 23,003 |
SUMA PTU D

SUNA PLS ee 328,00

ROZLICZENTE PLATNOSCI

Gotéuka 328,00
Wptacono razen 328,00
000255/0622 #450 2U

Nir Nabyy -
2022-07-28 | 3 21:02
A5B67ABE976FAB57435C4383BEF5145306370

Mn

Nr transakesi 013-001-000054 9420

DZTEKUJEMY 2A ZAKUPY
ZAPRASZANY PONOWNTE

www. Jeroymerlin.Ppl



## Nougat /donut

If you don't have a gpu better not use it :)



OCR model that parses straight into markdown format

https://github.com/facebookresearch/nougat

However it can hallucinate things

The following code is taken from https://huggingface.co/spaces/hf-vision/nougat-transformers

In [16]:
from PIL import Image
from nougat.dataset.rasterize import rasterize_paper
from transformers import NougatProcessor, VisionEncoderDecoderModel
import torch
import os

processor = NougatProcessor.from_pretrained("facebook/nougat-small")
model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-small")

In [18]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device) 
device

'cpu'

In [22]:
url = "https://www.africau.edu/images/default/sample.pdf"

r = requests.get(url, allow_redirects=True)

open('sample.pdf', 'wb').write(r.content)

3028

In [23]:
from pathlib import Path

def predict(image):
  # prepare PDF image for the model
  image = Image.open(image)
  pixel_values = processor(image, return_tensors="pt").pixel_values

  outputs = model.generate(
      pixel_values.to(device),
      min_length=1,
      max_new_tokens=1500,
      bad_words_ids=[[processor.tokenizer.unk_token_id]],
  )

  page_sequence = processor.batch_decode(outputs, skip_special_tokens=True)[0]
  page_sequence = processor.post_process_generation(page_sequence, fix_markdown=False)
  return page_sequence


def inference(pdf_file):
  file_name = pdf_file.name

  images = rasterize_paper(file_name, return_pil=True)
  sequence = ""
  # infer for every page and concat
  for image in images:
    sequence += predict(image)


  content = sequence.replace(r'\(', '$').replace(r'\)', '$').replace(r'\[', '$$').replace(r'\]', '$$')
  with open(f"{os.getcwd()}/output.md","w+") as f:
      f.write(content)
      f.close()

      
  return content, f"{os.getcwd()}/output.md"


inference(Path("sample.pdf"))



('\n\n## Appendix A Simple PDF File\n\nThis is a small demonstration .pdf file -\n\njust for use in the Virtual Mechanics tutorials. More text. And more text. And more text. And more text. And more text.\n\nAnd more text. And more text.\n\n\n\n## 6 Conclusion\n\nIn this thesis, we have presented a new method for computing the performance of the proposed method. We have presented a new method for computing the performance of the proposed method.\n\n',
 'c:\\Users\\kryst\\Documents\\PIMPS\\tutorials/output.md')

## Visual Question Answering / Document Question Answering

https://huggingface.co/impira/layoutlm-document-qa

These models are trained to retrieve information with understanding image layout

However, first we need to know which document to pick.

## Clip / Blip models

If we want to use natural language to search for images we can create captions for images

https://huggingface.co/Salesforce/blip-image-captioning-large

https://huggingface.co/spaces/pharmapsychotic/CLIP-Interrogator

